# Diffusion

In this notebook you will:
- Analyze one-dimensional diffusion
- Evolve a delta function initial condition
- Compare with analytic solution
- Investigate position-dependent diffusion constant: e.g. $D = D_0 \exp(- |x| / \lambda)$
- Try to solve this new problem analytically

In a nutshell:
- ...

For the future:
- ...

Resources:
- ...

In [2]:
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

# $\frac{\partial C}{\partial t} = D \frac{\partial^2 C}{\partial^2 x}$

# $\frac{\partial C}{\partial t} = \frac{\partial}{\partial x}(D \frac{\partial C}{\partial x})$